## Versão Local Projeto 3 Megadados
## Frederico e Raphael

In [1]:
import re
from graphframes import *
from pprint import pprint

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Load Data

In [28]:
non_decimal = re.compile(r'[^\d]+')
txt = sc.textFile("./mock.txt")
titles = sc.textFile("./mockTitles.txt")

In [29]:
titles, txt

(./mockTitles.txt MapPartitionsRDD[3179] at textFile at NativeMethodAccessorImpl.java:0,
 ./mock.txt MapPartitionsRDD[3177] at textFile at NativeMethodAccessorImpl.java:0)

In [30]:
titles.collect()

['Eh',
 'Quarta',
 'Feira',
 'Meus',
 'Bacanos',
 'Eae',
 'Galera',
 'Desse',
 'Brasil']

## Nodes

In [38]:
nodes = titles.zipWithIndex().map(lambda x: (x[1] + 1, x[0]))
pprint(nodes.collect())
nodesDf = nodes.toDF(['id', 'title'])

nodesDf.show(n=99)

[(1, 'Eh'),
 (2, 'Quarta'),
 (3, 'Feira'),
 (4, 'Meus'),
 (5, 'Bacanos'),
 (6, 'Eae'),
 (7, 'Galera'),
 (8, 'Desse'),
 (9, 'Brasil')]
+---+-------+
| id|  title|
+---+-------+
|  1|     Eh|
|  2| Quarta|
|  3|  Feira|
|  4|   Meus|
|  5|Bacanos|
|  6|    Eae|
|  7| Galera|
|  8|  Desse|
|  9| Brasil|
+---+-------+



## Edges

In [37]:
def edge_map(x):
    p = non_decimal.sub(' ', x).split(' ')
    return (p[0], p[1:])

edges = txt.map(edge_map).flatMapValues(lambda x: x)

edgesDf = edges.toDF(['src', 'dst'])
edgesDf.show(n=20)

+---+---+
|src|dst|
+---+---+
|  1|  2|
|  1|  3|
|  2|  1|
|  2|  3|
|  2|  4|
|  3|  1|
|  3|  2|
|  3|  2|
|  5|  3|
|  6|  2|
|  6|  8|
|  7|  1|
|  7|  2|
|  7|  3|
|  9|  3|
+---+---+



## Create graph from scratch

In [33]:
g = GraphFrame(nodesDf, edgesDf)

### Test graph properties

In [34]:
g.inDegrees.show(), g.outDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  3|       5|
|  8|       1|
|  1|       3|
|  4|       1|
|  2|       5|
+---+--------+

+---+---------+
| id|outDegree|
+---+---------+
|  7|        3|
|  3|        3|
|  5|        1|
|  6|        2|
|  9|        1|
|  1|        2|
|  2|        3|
+---+---------+



(None, None)

In [35]:
res = g.bfs('id=9', 'id=4')

In [36]:
res.show()

+-----------+------+----------+------+-----------+------+---------+
|       from|    e0|        v1|    e1|         v2|    e2|       to|
+-----------+------+----------+------+-----------+------+---------+
|[9, Brasil]|[9, 3]|[3, Feira]|[3, 2]|[2, Quarta]|[2, 4]|[4, Meus]|
|[9, Brasil]|[9, 3]|[3, Feira]|[3, 2]|[2, Quarta]|[2, 4]|[4, Meus]|
+-----------+------+----------+------+-----------+------+---------+



## Store parquet

In [29]:
def store_parquet(g):
    g.vertices.write.parquet("./vertices", mode='overwrite')
    g.edges.write.parquet("./edges", mode='overwrite')
    return True

In [30]:
store_parquet(g)

True

## Load Parquet

In [31]:
def load_parquet(vertices_path, edges_path):
    sameV = sqlContext.read.parquet(vertices_path)
    sameE = sqlContext.read.parquet(edges_path)
    sameG = GraphFrame(sameV, sameE)
    return sameG

g = load_parquet('./vertices/', './edges/')
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  7|       1|
|  3|       8|
|  5|       4|
|  6|       2|
|  9|       2|
|  1|       7|
| 12|       1|
| 13|       1|
|  2|      10|
+---+--------+

